# Creating Sender and Receiver Datasets

In [44]:
import pandas as pd 
import numpy as np
import math
from datetime import datetime
import joblib

In [2]:
data = pd.read_csv("realEmtHashed.csv")

In [3]:
data.head()

,userID,timestamp,payeeEmail,isp,latitude,longitude,money_amount,money_type
0,044f9d391a27b59859fb3b274237671ff246bb9b69ae7d...,2018-06-01 19:02:54.026-0400,2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703d1...,Cable Cable,44.53,-78.75,500.0,CAD
1,3ec92838d15518ea50355e7adfa01d470a7e49479c426d...,2018-06-01 05:37:11.480-0400,480e4ce89838a108880a7ba13475d9aa92e995476a86ea...,Primus Telecommunications Canada,43.89,-78.88,2571.0,CAD
2,0f772a1e33aec1998c7a917e5cf67f30eeb485db693c57...,2018-06-01 00:44:01.691-0400,02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a652...,Bell Canada,45.73,-73.71,1025.0,CAD
3,30e1bb678ca1426e122bdb953a2320abbacd14a1dd093b...,2018-06-01 00:33:22.304-0400,7e28bb7504342ed3806218195ab31d7dcab330cc7511ad...,Amanah Tech,43.65,-79.42,1510.0,CAD
4,5f1331872f036401287146d98e5b7f9d23a24a789b4d72...,2018-06-01 19:27:57.245-0400,42ad4701060ceaebd11b09360ca9a24d75d217f90921de...,Distributel Communications,45.55,-73.58,69.0,CAD


# Recatagorizing Data for Sender Accounts
## Changed the column headings to the sender account number with attributes about the sender account as the rows.

In [21]:
test_len = 10000
data_len = len(data)
new_data = {}
start_time = datetime.now()

for i in range(test_len): 
    row = data.iloc[i]
    sender = row['userID']
    amount = row['money_amount']
    location = (row['latitude'],row['longitude'])
    time = row['timestamp']
    ISP = row['isp']
    receiver = row['payeeEmail']
    
    if sender not in new_data: 
        new_data[sender] = {'transactions':1,'amounts':[],'locations':[],'receivers':{},'time_stamps':[],'senderISP':[]}
    else:
        new_data[sender]['transactions'] += 1
    
    new_data[sender]['amounts'].append(amount)
    new_data[sender]['time_stamps'].append(time)

    if location not in new_data[sender]['locations']:
        new_data[sender]['locations'].append(location)
        
    if ISP not in new_data[sender]['senderISP']:
        new_data[sender]['senderISP'].append(ISP)
        
    if receiver in new_data[sender]['receivers']:
        new_data[sender]['receivers'][receiver] += 1 
    else: 
        new_data[sender]['receivers'][receiver] = 1
    if i % 100000 == 0: 
        duration = datetime.now() - start_time
        print("Parsed "+ str(i) + " datapoints in: ", duration)

duration = datetime.now() - start_time
print("Parsed all data in: ", duration)

Parsed 0 datapoints in:  0:00:00.002009
Parsed all data in:  0:00:06.871806


# Saving New Sender Dataset

In [22]:
sender = pd.DataFrame.from_dict(new_data)

In [23]:
sender.head(15)

,044f9d391a27b59859fb3b274237671ff246bb9b69ae7d66db3a7a3875b9520d,3ec92838d15518ea50355e7adfa01d470a7e49479c426db3ffc65befe6a2e622,0f772a1e33aec1998c7a917e5cf67f30eeb485db693c57da470cf67d340c2453,30e1bb678ca1426e122bdb953a2320abbacd14a1dd093b77a4b2f33d84fc937e,5f1331872f036401287146d98e5b7f9d23a24a789b4d72c3731dcabfc67bbc48,a844f08602b27f68ecbe4685991c427948dc3ade39775ea3d29e99bc1c789c1d,800012ea1c431863b68eff6dbf714090466718395a6158b76e62b9e16b63d6f8,01fad62c3e5b6cbf95a26d59abd3249796e56a1caa027b013b470e06ffc4afba,8b11e3e97273699aeca08dbb84f884921c53f7af83144143166db368cb1b7df7,831b7e058c9542cea19d2fecbebf0e838ee5a0ccaabe06e9b13eab2940cd9c0f,...,e5c5ac9bf2d3b1b114043affb014ad06b1e50b9260ed033151afb18c26f499f6,8b566b63fef7abd0f0dcc8922843e912785d874fe04d1b24940041e783352539,4d42f17b6c4d47cf80eae6a320b95a085b432072872d231bb622e6f95648a58d,756c9da3bf360f62768fb6b7e773ed34225258114e6d098cbdf66d62011e63e9,7ac76c06631a5afc4653c863e5b54bf6c8c1c373c1cefc7c621878ced76d727e,be48cb3dd8e3c86de691d529f629389d644c58edd95ff2c0e3a303885eaf869f,5a235665dc8c09a3eb1de698157d411f58c0ececc41416959a421441e639f1fe,9b77150c5a7ef68ca2f11936739540e99ccb3ce0c21fe594f36ea48091c4c59b,6af6f566da1e80d797352fc9f17e1cff9218d0e8b49932020da1a85424f54822,c7be6a75a5cfb169e57422f4833933b3e8c4399e2ace222b86ce03f80046899e
amounts,[500.0],[2571.0],[1025.0],[1510.0],[69.0],[820.0],[666.66],[350.0],[2776.57],[200.0],...,[207.9],[480.0],[845.0],[300.0],[925.0],[1050.0],[1550.0],[3000.0],[179.0],[50.0]
locations,"[(44.53, -78.75)]","[(43.89, -78.88)]","[(45.73, -73.71)]","[(43.65, -79.42)]","[(45.55, -73.58)]","[(50.05, -119.29)]","[(48.48, -123.37)]","[(45.33, -75.67)]","[(49.28, -123.11)]","[(49.92, -96.96)]",...,"[(53.91, -122.75)]","[(51.08, -113.96)]","[(45.44, -73.46)]","[(49.93, -105.97)]","[(49.26, -123.16)]","[(43.87, -79.44)]","[(49.19, -122.94)]","[(51.03, -113.82)]","[(43.65, -79.48)]","[(43.78, -79.42)]"
receivers,{'2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703...,{'480e4ce89838a108880a7ba13475d9aa92e995476a86...,{'02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a6...,{'7e28bb7504342ed3806218195ab31d7dcab330cc7511...,{'42ad4701060ceaebd11b09360ca9a24d75d217f90921...,{'a81fd73419341dda2674163e8c5916943968d2857aa8...,{'922efec678a4b9ef3feb83cfa94c2cb3328864a3ea4c...,{'bf6f30e2e6e31010cfcd15eb765b7a4cfc21ff33fe29...,{'99802f30b2029bcc495725c61b51464a4bd5a85ccbf3...,{'f2717c125ac73191416ed910bee13fbaaeb1c90d8a7b...,...,{'ff19617e9adbab696de747eec0be5c092929ebcdff25...,{'291c9c7d6e88da402d21bba58d9e0ed2db0d542f1b11...,{'d667389426be58e330cd61b301a71f4e85e9bec57ede...,{'798c4f48f2a41b51f3b790035465db3c790c82b3dbac...,{'5a5a7451fa52fb233c268b9d02000770659f52a5aa67...,{'5823f965be7adcaaffaf898245f879229a1f38b316b6...,{'b510dffd23ec87b55012a779ab2be3de8086ce2e653f...,{'cc3778e3b198541c67478b0332fa8a77112d0497c0a0...,{'8772b440ab2fa81ce2a27a0a0f3f71d342d016d9de78...,{'854e7667a9b0a2b0cee86d0eb2f4bea01dd1a35a7c51...
senderISP,[Cable Cable],[Primus Telecommunications Canada],[Bell Canada],[Amanah Tech],[Distributel Communications],[Shaw Communications],[Shaw Communications],[Shared Services Canada],[Bell Canada],[Shaw Communications],...,[Shaw Communications],[CIK Telecom],[ACN DSL],[SaskTel],[Telus Communications],[York Region District School Board],[Telus Communications],[Telus Communications],[Bell Canada],[Bell Canada]
time_stamps,[2018-06-01 19:02:54.026-0400],[2018-06-01 05:37:11.480-0400],[2018-06-01 00:44:01.691-0400],[2018-06-01 00:33:22.304-0400],[2018-06-01 19:27:57.245-0400],[2018-06-01 00:12:43.788-0400],[2018-06-01 11:05:43.082-0400],[2018-06-01 14:00:51.380-0400],[2018-06-01 15:52:35.582-0400],[2018-06-01 08:22:02.319-0400],...,[2018-06-01 16:05:34.136-0400],[2018-06-01 18:26:56.105-0400],[2018-06-01 06:35:16.467-0400],[2018-06-01 12:46:47.714-0400],[2018-06-01 13:34:44.598-0400],[2018-06-01 09:22:47.131-0400],[2018-06-01 10:41:38.070-0400],[2018-06-01 08:50:27.956-0400],[2018-06-01 14:40:53.559-0400],[2018-06-01 07:54:21.376-0400]
transactions,1,1,1,1,1,1,1,1,

In [24]:
def unique_senders(dataset): 
    different_accounts = [] 
    for name in dataset.iteritems(): 
        value = name[1]['receivers']
        different_accounts.append(len(value.keys()))
    return different_accounts

In [25]:
def unique_receivers(dataset): 
    different_accounts = [] 
    for name in dataset.iteritems(): 
        value = name[1]['senders']
        different_accounts.append(len(value.keys()))
    return different_accounts

In [26]:
def mean_amount(dataset):
    mean_amount = []   

    #Mean transaction amount per account ID
    for name in dataset.iteritems():
        value = name[1]['amounts']
        amount = 0 
        bad_values = 0
        #Using a loop and not sum because the some of the fucks in here are strings
        for item in value:
            try:
                amount += float(item)
            except ValueError: 
                bad_values += 1 

        transaction_number = len(value) - bad_values

        if transaction_number != 0:
            mean = amount/transaction_number
            mean_amount.append(mean)
        else: 
            mean_amount.append(0)
    return mean_amount

In [27]:
def stnd_dev(dataset):
    #Standard Deviation for amounts 
    std_dev = []
    z_score = [] 
    for name in dataset.iteritems(): 
        value = name[1]['amounts']
        mean = name[1]['mean_transaction_amount']
        variance = 0 
        bad_values = 0 
        for item in value: 
            try: 
                variance += math.pow((float(item) - mean),2)
            except: 
                bad_values += 1

        transaction_number = len(value) - bad_values 
        if transaction_number != 0: 
            std = math.sqrt(variance)/transaction_number
            std_dev.append(std)
        else: 
            std_dev.append(0)
    return std_dev

In [28]:
def ISP_changes(dataset):
    #Number of Different ISP's 
    ISP_changes = [] 

    for name in dataset.iteritems(): 
        value = name[1]['senderISP']
        ISP_changes.append(len(value))
    return ISP_changes

In [29]:
def timeDeltas(dates):
    '''
    Given a list of dates of the form %Y-%m-%d %H:%M:%S.%f%z return an array of datetime.timedelta
    objects that corresponds to the differences in time  between consecutive days in the dates array
    '''
    
    # convert to datetime objects
    datetimes = []
    for date in dates:
        datetimes.append(datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f%z"))
     
    # sort chronologically
    datetimes.sort()
    
    dateDeltas = np.diff(datetimes)
        
    return dateDeltas



def trans_freq(dataset):
    '''
    Get mean time between transactions and std dev for all accounts, as well as frequency
    
    returns [mean_trans_times, trans_times_std, freq]
    '''
    
    mean_trans_times = []
    trans_times_std = []
    freq = []

    # loop through all accounts
    for name in dataset.iteritems(): 

        # get transaction time stamps 
        transaction_times = name[1]['time_stamps']

        if len(transaction_times) >= 2:
            # get array of times between consecutive time stamps (in days)
            times_between_trans = timeDeltas(transaction_times)
            dateToDays = np.vectorize(lambda t : t.total_seconds()/86400)
            times_between_trans = dateToDays(times_between_trans)


            # average time between transactions (in days), and std dev
            avg_time = np.mean(times_between_trans) 
            std_time = np.std(times_between_trans)

            mean_trans_times.append(avg_time)
            trans_times_std.append(std_time)
            freq.append(1/avg_time)

        else:
            mean_trans_times.append(0)
            trans_times_std.append(0)
            freq.append(0)




    return mean_trans_times, trans_times_std, freq


In [30]:
def distance_from_us(dataset):
    #Distance from the American Border
    distance = []

    for name in dataset.iteritems(): 
        value = name[1]['locations'][0]
        y,x = value

        if x >= -125.680 and x <= -95.190 or x < -125.680: 
            border = 49.00 

        elif x > -95.190 and x <= -82.840: 
            border = -0.591*x - 7.267

        elif x > -82.840 and x <= -68.912: 
            border = 0.402*x + 75

        elif x > -68.912 and x <= -66.890: 
            border = -2.850*x - 149.159

        elif x > -95.19 and x <= -82.84: 
            border = -0.591*x - 7.267

        else: 
            border = 44.7 

        distance.append(abs(y - border))
        
    dataset.loc['distance_from_border'] = distance
    # Replacing all the nan values with the average
    count = 0
    total = 0 

    for name in dataset.iteritems():
        value = name[1]['distance_from_border']
        if not np.isnan(value):
            count += 1
            total += value

    average = total/count

    for i in range(len(distance)): 
        if np.isnan(distance[i]): 
            distance[i] = average

    return distance

In [31]:
def time_between(dataset): 
    number_of_transactions = []
    for name in dataset.iteritems(): 
        value = name[1]['time_stamps']
        bad = 0 
        for date in value:
            date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f%z")
            if date.hour > 22 or date.hour < 6: 
                bad += 1
                
        number_of_transactions.append(bad)
    
    return number_of_transactions

In [32]:
def major_cities(dataset):
    final = [] 
    for name in dataset.iteritems(): 
        value = name[1]['locations'][0]
        y1,x1 = value
        cities = {'Toronto':(43.65, 79.38),'Calgary':(51.04, 114.07),'Vancouver':(49.28, 123.12),
                  'Montreal':(45.50, 73.56), 'Winnipeg':(49.89, 97.13), 'Quebec City':(45.42, 75.69),
                  'Regina':(50.44, 104.61),'Saskatoon':(52.13, 106.67),'Edmonton':(53.54, 113.49),
                  'St Johns':(47.56, 52.71),'Charlottetown':(46.23, 63.13),'Halifax':(44.64, 63.57),
                  'Fredericton':(45.96, 66.64)}
        
        distances = []
        for location in cities.keys(): 
            y2,x2 = cities[location]
            dist = np.abs((y2-y1))
            distances.append(dist)
        
        final.append(min(distances))
        
    for i in range(len(final)): 
        if np.isnan(final[i]):
            final[i] = 0.2
            
    return final

# Features For Sender Dataset

In [33]:
# linear runtime
import time
s = time.time()
sender.loc['mean_transaction_amount'] = mean_amount(sender)
sender.loc['transacton_std_dev'] = stnd_dev(sender)
sender.loc['total_ISPs'] = ISP_changes(sender)
sender.loc['mean_trans_times'] = trans_freq(sender)[0]
sender.loc['trans_times_std'] = trans_freq(sender)[1]
sender.loc['transaction_frequency'] = trans_freq(sender)[2]
sender.loc['distance_from_border'] = distance_from_us(sender)
sender.loc['unique_receivers'] = unique_senders(sender)
sender.loc['night_time'] = time_between(sender)
sender.loc['major_city'] = major_cities(sender)
#Total Transactions is also a feature included
print(time.time()-s)

13.770509004592896


In [41]:
sender.head(15)

,044f9d391a27b59859fb3b274237671ff246bb9b69ae7d66db3a7a3875b9520d,3ec92838d15518ea50355e7adfa01d470a7e49479c426db3ffc65befe6a2e622,0f772a1e33aec1998c7a917e5cf67f30eeb485db693c57da470cf67d340c2453,30e1bb678ca1426e122bdb953a2320abbacd14a1dd093b77a4b2f33d84fc937e,5f1331872f036401287146d98e5b7f9d23a24a789b4d72c3731dcabfc67bbc48,a844f08602b27f68ecbe4685991c427948dc3ade39775ea3d29e99bc1c789c1d,800012ea1c431863b68eff6dbf714090466718395a6158b76e62b9e16b63d6f8,01fad62c3e5b6cbf95a26d59abd3249796e56a1caa027b013b470e06ffc4afba,8b11e3e97273699aeca08dbb84f884921c53f7af83144143166db368cb1b7df7,831b7e058c9542cea19d2fecbebf0e838ee5a0ccaabe06e9b13eab2940cd9c0f,...,e5c5ac9bf2d3b1b114043affb014ad06b1e50b9260ed033151afb18c26f499f6,8b566b63fef7abd0f0dcc8922843e912785d874fe04d1b24940041e783352539,4d42f17b6c4d47cf80eae6a320b95a085b432072872d231bb622e6f95648a58d,756c9da3bf360f62768fb6b7e773ed34225258114e6d098cbdf66d62011e63e9,7ac76c06631a5afc4653c863e5b54bf6c8c1c373c1cefc7c621878ced76d727e,be48cb3dd8e3c86de691d529f629389d644c58edd95ff2c0e3a303885eaf869f,5a235665dc8c09a3eb1de698157d411f58c0ececc41416959a421441e639f1fe,9b77150c5a7ef68ca2f11936739540e99ccb3ce0c21fe594f36ea48091c4c59b,6af6f566da1e80d797352fc9f17e1cff9218d0e8b49932020da1a85424f54822,c7be6a75a5cfb169e57422f4833933b3e8c4399e2ace222b86ce03f80046899e
amounts,[500.0],[2571.0],[1025.0],[1510.0],[69.0],[820.0],[666.66],[350.0],[2776.57],[200.0],...,[207.9],[480.0],[845.0],[300.0],[925.0],[1050.0],[1550.0],[3000.0],[179.0],[50.0]
locations,"[(44.53, -78.75)]","[(43.89, -78.88)]","[(45.73, -73.71)]","[(43.65, -79.42)]","[(45.55, -73.58)]","[(50.05, -119.29)]","[(48.48, -123.37)]","[(45.33, -75.67)]","[(49.28, -123.11)]","[(49.92, -96.96)]",...,"[(53.91, -122.75)]","[(51.08, -113.96)]","[(45.44, -73.46)]","[(49.93, -105.97)]","[(49.26, -123.16)]","[(43.87, -79.44)]","[(49.19, -122.94)]","[(51.03, -113.82)]","[(43.65, -79.48)]","[(43.78, -79.42)]"
receivers,{'2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703...,{'480e4ce89838a108880a7ba13475d9aa92e995476a86...,{'02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a6...,{'7e28bb7504342ed3806218195ab31d7dcab330cc7511...,{'42ad4701060ceaebd11b09360ca9a24d75d217f90921...,{'a81fd73419341dda2674163e8c5916943968d2857aa8...,{'922efec678a4b9ef3feb83cfa94c2cb3328864a3ea4c...,{'bf6f30e2e6e31010cfcd15eb765b7a4cfc21ff33fe29...,{'99802f30b2029bcc495725c61b51464a4bd5a85ccbf3...,{'f2717c125ac73191416ed910bee13fbaaeb1c90d8a7b...,...,{'ff19617e9adbab696de747eec0be5c092929ebcdff25...,{'291c9c7d6e88da402d21bba58d9e0ed2db0d542f1b11...,{'d667389426be58e330cd61b301a71f4e85e9bec57ede...,{'798c4f48f2a41b51f3b790035465db3c790c82b3dbac...,{'5a5a7451fa52fb233c268b9d02000770659f52a5aa67...,{'5823f965be7adcaaffaf898245f879229a1f38b316b6...,{'b510dffd23ec87b55012a779ab2be3de8086ce2e653f...,{'cc3778e3b198541c67478b0332fa8a77112d0497c0a0...,{'8772b440ab2fa81ce2a27a0a0f3f71d342d016d9de78...,{'854e7667a9b0a2b0cee86d0eb2f4bea01dd1a35a7c51...
senderISP,[Cable Cable],[Primus Telecommunications Canada],[Bell Canada],[Amanah Tech],[Distributel Communications],[Shaw Communications],[Shaw Communications],[Shared Services Canada],[Bell Canada],[Shaw Communications],...,[Shaw Communications],[CIK Telecom],[ACN DSL],[SaskTel],[Telus Communications],[York Region District School Board],[Telus Communications],[Telus Communications],[Bell Canada],[Bell Canada]
time_stamps,[2018-06-01 19:02:54.026-0400],[2018-06-01 05:37:11.480-0400],[2018-06-01 00:44:01.691-0400],[2018-06-01 00:33:22.304-0400],[2018-06-01 19:27:57.245-0400],[2018-06-01 00:12:43.788-0400],[2018-06-01 11:05:43.082-0400],[2018-06-01 14:00:51.380-0400],[2018-06-01 15:52:35.582-0400],[2018-06-01 08:22:02.319-0400],...,[2018-06-01 16:05:34.136-0400],[2018-06-01 18:26:56.105-0400],[2018-06-01 06:35:16.467-0400],[2018-06-01 12:46:47.714-0400],[2018-06-01 13:34:44.598-0400],[2018-06-01 09:22:47.131-0400],[2018-06-01 10:41:38.070-0400],[2018-06-01 08:50:27.956-0400],[2018-06-01 14:40:53.559-0400],[2018-06-01 07:54:21.376-0400]
transactions,1,1,1,1,1,1,1,1,

# Saving as a joblib file

In [20]:
joblib.dump(sender,'sender_data')

['sender_data']

# Recatagorizing Data for Receiver Emails
## Changed the column headings to the sender account number with attributes about the Receiver Email as the rows.

In [45]:
test_len = 10000
data_len = len(data)
new_data = {}
start_time = datetime.now()

for i in range(data_len): 
    row = data.iloc[i]
    money_sender = row['userID']
    amount = row['money_amount']
    location = (row['latitude'],row['longitude'])
    time = row['timestamp']
    ISP = row['isp']
    receiver = row['payeeEmail']
    
    if receiver not in new_data: 
        new_data[receiver] = {'transactions':1,'amounts':[],'senders':{},'time_stamps':[]}
    else:
        new_data[receiver]['transactions'] += 1
    
    new_data[receiver]['amounts'].append(amount)
    new_data[receiver]['time_stamps'].append(time)
        
    if money_sender in new_data[receiver]['senders']:
        new_data[receiver]['senders'][money_sender] += 1 
    else: 
        new_data[receiver]['senders'][money_sender] = 1
    
    if i % 100000 == 0: 
        duration = datetime.now() - start_time
        print("Parsed "+ str(i) + " datapoints in: ", duration)

duration = datetime.now() - start_time
print("Parsed data in: ", duration)

Parsed 0 datapoints in:  0:00:00.001731
Parsed 100000 datapoints in:  0:00:43.352201
Parsed 200000 datapoints in:  0:01:33.487506
Parsed 300000 datapoints in:  0:02:16.254231
Parsed 400000 datapoints in:  0:02:58.409365
Parsed 500000 datapoints in:  0:03:43.199345
Parsed 600000 datapoints in:  0:04:26.011074
Parsed 700000 datapoints in:  0:05:09.173240
Parsed 800000 datapoints in:  0:05:52.064882
Parsed 900000 datapoints in:  0:06:39.462647
Parsed 1000000 datapoints in:  0:07:21.447981
Parsed 1100000 datapoints in:  0:08:04.103585
Parsed 1200000 datapoints in:  0:08:46.673276
Parsed 1300000 datapoints in:  0:09:28.262806
Parsed 1400000 datapoints in:  0:10:10.099081
Parsed 1500000 datapoints in:  0:10:51.932096
Parsed 1600000 datapoints in:  0:11:32.167299
Parsed 1700000 datapoints in:  0:12:11.643218
Parsed 1800000 datapoints in:  0:12:53.491828
Parsed 1900000 datapoints in:  0:13:33.550955
Parsed 2000000 datapoints in:  0:14:15.065820
Parsed 2100000 datapoints in:  0:14:57.319817
Par

In [46]:
receiver = pd.DataFrame.from_dict(new_data)

# Features For Receiver Dataset

In [49]:
start_time = datetime.now()

receiver.loc['mean_transaction_amount'] = mean_amount(receiver)
receiver.loc['transacton_std_dev'] = stnd_dev(receiver)

receiver.loc['transaction_frequency'] = trans_freq(receiver)[2]
duration = datetime.now() - start_time
print('Time Elapsed: ', duration)

receiver.loc['unique_senders'] = unique_receivers(receiver)
receiver.loc['night_time'] = time_between(receiver)
receiver.loc['mean_trans_times'] = trans_freq(receiver)[0]

duration = datetime.now() - start_time
print('Time Elapsed: ', duration)

receiver.loc['trans_times_std'] = trans_freq(receiver)[1]
receiver.loc['transaction_frequency'] = trans_freq(receiver)[2]
#Total Transactions also a feature which is included
duration = datetime.now() - start_time
print('Total Time Elapsed: ', duration)

Time Elapsed:  0:24:31.370236
Time Elapsed:  0:53:33.609543
Total Time Elapsed:  1:22:54.176289


In [50]:
receiver.head(15)

,2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703d110a400db4053f82f74,480e4ce89838a108880a7ba13475d9aa92e995476a86ea093ec85ba9561bff19,02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a6523e023b951f8ccd424f,7e28bb7504342ed3806218195ab31d7dcab330cc7511ad7e99470d35faf27097,42ad4701060ceaebd11b09360ca9a24d75d217f90921deecae5fe2238750f76c,a81fd73419341dda2674163e8c5916943968d2857aa829450c1177858ebcec6c,922efec678a4b9ef3feb83cfa94c2cb3328864a3ea4c6fb4e2b252340eb8949d,bf6f30e2e6e31010cfcd15eb765b7a4cfc21ff33fe295b2629a1f7aa7560a348,99802f30b2029bcc495725c61b51464a4bd5a85ccbf3080468d4684f69b99ded,f2717c125ac73191416ed910bee13fbaaeb1c90d8a7b2d789aeeafc854fddb67,...,0d8ee3425fd2ad48cc8fd22e236052d22150db2ae953b1f7992a55aeeb2e279e,6bcfd30e37b050170b19a0acd4b91f8bb8a05affea8cdc6bb92b524da4dc19c7,87283b531005bb16d1bff927648d38d143cf21d2a09aa4a9cefdb47d1e7d4d1e,606b5bf23b9b8001fc9717c97224b06cbb01a2108cccd30087da230460f9b9a6,18e7f1f03c26fa1f6b821a43928042b4caf81342fde77526b02a959f17c523b1,b7f82d880e7469add556fa93d50810474e688f0dcf0e7bb9770cef27ea5299d4,a43632656882d7515f7a5445c982d5077f2a007502133744eea1869e8c806740,25bffa2f0bc780e3ee52534a1b1b51b7c100d88e10132ca04ef4e7489f23bfa2,d5978bf81a7e999dc4ec2acf5eae095c03bf0ef1390fbfa0ea0d80ccdb001e14,eab2cd9a6a6bd5c18862b911dd7cef59bd4a801c785fefa1e235a020aee7a310
amounts,[500.0],"[2571.0, 2571.0, 2571.0, 2571.0, 2571.0, 2571....","[1025.0, 1025.0, 1025.0, 1025.0, 1025.0, 500.0...","[1510.0, 1510.0]","[69.0, 469.0, 69.0, 69.0, 473.0, 69.0, 69.0, 6...","[820.0, 820.0, 820.0, 820.0, 820.0, 500.0, 820...","[666.66, 330.0, 615.73, 250.0]","[350.0, 350.0, 315.0, 350.0, 680.0, 350.0, 350...","[2776.57, 2742.65, 30.0, 776.57, 776.57, 2000....",[200.0],...,[100.0],[20.0],[100.0],[224.87],[1000.0],[200.0],[500.0],[900.0],[500.0],[20.0]
senders,{'044f9d391a27b59859fb3b274237671ff246bb9b69ae...,{'3ec92838d15518ea50355e7adfa01d470a7e49479c42...,{'0f772a1e33aec1998c7a917e5cf67f30eeb485db693c...,{'30e1bb678ca1426e122bdb953a2320abbacd14a1dd09...,{'5f1331872f036401287146d98e5b7f9d23a24a789b4d...,{'a844f08602b27f68ecbe4685991c427948dc3ade3977...,{'800012ea1c431863b68eff6dbf714090466718395a61...,{'01fad62c3e5b6cbf95a26d59abd3249796e56a1caa02...,{'8b11e3e97273699aeca08dbb84f884921c53f7af8314...,{'831b7e058c9542cea19d2fecbebf0e838ee5a0ccaabe...,...,{'a1a6506a2ec1d7a42117fdbf34ad521f4191a7094ea3...,{'71e34674800c09cf437327f2fe5678da07d6015a4b41...,{'810be5f0a72a2e79127ef47d0ba7e48731a29d6dc760...,{'81d730e1c5fa7033fec80349c8f7dbb6d72966fa60cd...,{'d9291eb7de5f0d089c48ccac57cc3fe66493cdbbb6e2...,{'fb125d26fff3a473da9430ea0a77555d2977f6c3786e...,{'aa07cf0e58f300caf64ec51c96ca0732a49cd2050ddd...,{'1642b76e70a38bed51b72ba26102fdec6f42dee22e0b...,{'02e5ce0630bf03b7a491c7cde335e08dd14358a72da6...,{'1b92b012dc62df94bc58ce2d403a5fbb687fcae1589b...
time_stamps,[2018-06-01 19:02:54.026-0400],"[2018-06-01 05:37:11.480-0400, 2018-03-01 09:4...","[2018-06-01 00:44:01.691-0400, 2018-04-30 10:5...","[2018-06-01 00:33:22.304-0400, 2018-03-23 09:1...","[2018-06-01 19:27:57.245-0400, 2017-12-01 06:1...","[2018-06-01 00:12:43.788-0400, 2017-12-01 19:5...","[2018-06-01 11:05:43.082-0400, 2018-04-30 22:2...","[2018-06-01 14:00:51.380-0400, 2018-06-15 15:4...","[2018-06-01 15:52:35.582-0400, 2018-05-01 13:4...",[2018-06-01 08:22:02.319-0400],...,[2017-08-31 23:53:10.725-0400],[2017-08-31 23:56:52.859-0400],[2017-08-31 23:59:49.201-0400],[2017-08-31 23:35:36.215-0400],[2017-08-31 23:54:33.430-0400],[2017-08-31 23:54:47.951-0400],[2017-08-31 23:37:28.421-0400],[2017-08-31 23:18:16.956-0400],[2017-08-31 23:04:53.080-0400],[2017-08-31 23:29:11.151-0400]
transactions,1,7,10,2,9,11,4,12,26,1,...,1,1,1,1,1,1,1,1,1,1
mean_transaction_amount,500,2571,922.5,1510,213.889,743.636,465.597,357.083,939.639,200,...,100,20,100,224.87,1000,200,500,900,500,20
transacton_std_dev,0,0,64.8508,0,68.8782,50.5061,89.385,32.6623,153.995,0,...,0,0,0,0,0,0,0,0,0,0
transaction_frequency,0,0.0403514,0.0373665,0.0143604,0.0334063,0.0330789,0.0197931,0.0827287,0.0886622,0,...,0,0,

In [51]:
joblib.dump(receiver,'receiver_data')

['receiver_data']